## Cross-Bucket Tests on all 3 Fragment Types using Kalman Filter

### Part A: (1) Import Fragment Data from MATLAB, (2) Save Them in a .pickle file, and (3) Load Them In

###### (HOWEVER, YOU DON'T NEED TO LOAD THEM IN AGAIN SINCE STEP 1 ALREADY DOES THAT FOR YOU!)

In [2]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import standard package ###
import numpy as np
from scipy import io
import sys

# Specify Fragment Types to be used for this Anaylsis 
frag_type = ['AD', 'HV', 'VM'] # ['AccDec', 'HillValley', 'VelMin']

In [2]:
 ## (OLD) Import Data ## 
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')
   
    outputX = "sortX_"+frag_type[i]
    locals()[outputX] = io.loadmat(folder + outputX + '.mat')

    outputY = "sortY_"+frag_type[i]
    locals()[outputY] = io.loadmat(folder + outputY + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[outputX] = np.squeeze(list(locals()[outputX].values())[3])
    locals()[outputY] = np.squeeze(list(locals()[outputY].values())[3])

In [15]:
## (OLD) Save Data in .pickle files ## 
import pickle

data_folder='/Users/rbhatt6/Documents/MATLAB/' # Folder where you want to save the data

for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    outputX = "sortX_"+frag_type[i]
    outputY = "sortY_"+frag_type[i]

    with open(data_folder + input + '.pickle','wb') as f:
        pickle.dump(locals()[input],f)

    with open(data_folder + outputX + '.pickle','wb') as f:
        pickle.dump(locals()[outputX],f)

    with open(data_folder + outputY + '.pickle','wb') as f:
        pickle.dump(locals()[outputY],f)

In [3]:
## (NEW) Import Data ##
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')

    output = "sortOut_" + frag_type[i]
    locals()[output] = io.loadmat(folder + output + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[output] = np.squeeze(list(locals()[output].values())[3])

### Part B: Preprocessing Decoder Data

In [6]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.metrics import get_R2_parts

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [4]:
# (NEW)
#For the Kalman filter, we use the position, velocity, and acceleration as outputs.
#Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            vel_X = float(output[j][0][k])
            vel_Y = float(output[j][1][k])
            acc_X = float(output[j][2][k])
            acc_Y = float(output[j][3][k])
            pos_X = float(output[j][4][k])
            pos_Y = float(output[j][5][k])
            temp = [vel_X, vel_Y, acc_X, acc_Y, pos_X, pos_Y]
            temp2.append(np.array(temp))
            #locals()[decoder_output][j][k].append(np.array(temp))
        #locals()[decoder_output][j].append(np.array(temp2))
        locals()[decoder_output].append(np.array(temp2))
        #temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))


In [ ]:
decoder_output_HV
decoder_output_HV[0].shape
decoder_output_HV[0]
decoder_output_HV[0][0]

In [272]:
# (OLD)
#For the Kalman filter, we use the position, velocity, and acceleration as outputs.
#Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    
    # Pulling local variables into new, temp variables
    outputX = locals()["sortX_"+frag_type[i]]
    outputY = locals()["sortY_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(outputX)):
        # Determine X and Y velocities 
        vel_X = (np.diff(outputX[j], axis = 0))      
        vel_X = np.concatenate((vel_X,vel_X[-1:,:]),axis=0)
        vel_Y = (np.diff(outputY[j], axis = 0))      
        vel_Y = np.concatenate((vel_Y,vel_Y[-1:,:]),axis=0)

        # Determine X and Y accelerations 
        acc_X = (np.diff(vel_X, axis = 0))   
        acc_X = np.concatenate((acc_X,acc_X[-1:,:]),axis=0)
        acc_Y = (np.diff(vel_Y, axis = 0))      
        acc_Y = np.concatenate((acc_Y,acc_Y[-1:,:]),axis=0)

        # Concatenating all 3 kinematics for X and Y components
        # Corrected decoder outputs (format = x,y vel, x,y acc, x,y pos)
        temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))

        locals()[decoder_output].append(temp)

### Part C: Partitioning and Running the Kalman Filter on the Same Buckets 

In [7]:
#Set what part of data should be part of the training/testing/validation sets

training_range=[0, 0.8]
valid_range=[0.8,0.9]
testing_range=[0.9, 1]

In [8]:
# Running decoder on X and Y components together to get the prediction nom and denom to later compute the combined XY_FVAF
from Neural_Decoding.runModelsKF import run_model_kf

for i in range(len(frag_type)):

    # Pulling local variables 
    input = locals()["sortIn_"+frag_type[i]]
    output = locals()["decoder_output_"+frag_type[i]]

    # Creating a local variable to hold (X, Y) predicted outputs for each frag type 
    parts = "pred_parts_" + frag_type[i]
    locals()[parts] = run_model_kf(input, output, training_range, testing_range, valid_range, "parts")

/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:48: RuntimeWarning: invalid value encountered in true_divide
  X_train=(X_train-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:49: RuntimeWarning: divide by zero encountered in true_divide
  X_test=(X_test-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:49: RuntimeWarning: invalid value encountered in true_divide
  X_test=(X_test-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:50: RuntimeWarning: invalid value encountered in true_divide
  X_valid=(X_valid-X_train_mean)/X_train_std


In [9]:
# Compute combined XY_FVAF (velocity only)
from Neural_Decoding.metrics import compute_XY_FVAF

for i in range(len(frag_type)):

    # Pulling previously computed predicted_parts (i.e. nom and denom)
    parts = locals()[ "pred_parts_" + frag_type[i]]
    
    # Creating a local variable to hold XY_FVAFs each frag type 
    XY_FVAF = "XY_FVAF_" + frag_type[i]
    locals()[XY_FVAF] = []

    for j in range(len(parts)):
        #curr_bucket = Kalman_R2s_combined[i]
        vel_x_nom = parts[j][0][0] # dim = (curr_bucket, nom, x_vel)
        vel_x_denom = parts[j][1][0] # dim = (curr_bucket, denom, x_vel)
        vel_y_nom = parts[j][0][1] # dim = (curr_bucket, nom, y_vel)
        vel_y_denom = parts[j][1][1] # dim = (curr_bucket, denom, y_vel)

        curr_bucket_XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
        locals()[XY_FVAF].append(curr_bucket_XY_FVAF)

In [17]:
XY_FVAF_HV

[0.11312576051669809,
 nan,
 0.3009460206465555,
 0.17672436958159832,
 0.3103356178769259,
 0.2847933442835944,
 0.15725940333019095,
 0.20430512492659825,
 0.0555907420103362,
 0.08908951041409252,
 0.23587576205121497,
 0.12279699280365264,
 0.25135105868315133,
 0.14666016348559596,
 0.1346890712506058,
 0.1115590403380633]

In [14]:
decoder_output_HV
decoder_output_HV[1].shape
decoder_output_HV[1]
decoder_output_HV[1][0]

[array([[ 5.32435091e-01, -1.25558256e-01,  8.29243385e-03,
          3.26141647e-03,  2.94288616e+01,  2.12332972e+02],
        [ 6.76339848e-01, -3.08129850e-02,  1.96040793e-03,
          3.60351340e-03,  4.49458890e+01,  2.10410463e+02],
        [ 5.98649981e-01,  4.21008543e-02, -7.39084234e-03,
          2.56843815e-03,  6.13307581e+01,  2.10636868e+02],
        ...,
        [ 2.90269123e-01, -2.35700044e-02,  7.82576622e-03,
          5.59795393e-03,  1.32371505e+02,  2.03465614e+02],
        [ 3.71531305e-01,  3.63904941e-02,  2.57149312e-04,
         -1.00104409e-03,  1.41047328e+02,  2.04020941e+02],
        [ 3.31851599e-01, -3.23826268e-02, -3.52585113e-03,
         -3.21702113e-03,  1.50005854e+02,  2.04134719e+02]]),
 array([[ 5.74221062e-03,  1.06509514e-01,  3.27925477e-05,
          2.87395140e-03,  4.09326252e+01,  2.31689859e+02],
        [ 5.82637463e-02,  1.31993177e-01,  3.42250406e-03,
         -2.95599777e-04,  4.15747141e+01,  2.34849675e+02],
        [ 1.04368

(2874, 6)

array([[ 5.74221062e-03,  1.06509514e-01,  3.27925477e-05,
         2.87395140e-03,  4.09326252e+01,  2.31689859e+02],
       [ 5.82637463e-02,  1.31993177e-01,  3.42250406e-03,
        -2.95599777e-04,  4.15747141e+01,  2.34849675e+02],
       [ 1.04368223e-01,  1.13737991e-01, -1.42902374e-03,
        -8.06027006e-04,  4.38987996e+01,  2.37933708e+02],
       ...,
       [ 3.71820483e-01,  7.53988446e-01,  2.05856350e-03,
        -6.82282163e-03,  2.94690542e+01,  2.48651217e+02],
       [ 4.34880313e-01,  5.75902123e-01,  2.38863696e-03,
        -6.94820331e-03,  3.95748921e+01,  2.65168730e+02],
       [ 4.50420499e-01,  3.91165834e-01, -1.56815554e-03,
        -7.44559769e-03,  5.08673478e+01,  2.77197248e+02]])

array([5.74221062e-03, 1.06509514e-01, 3.27925477e-05, 2.87395140e-03,
       4.09326252e+01, 2.31689859e+02])

### Part D: Partitioning and Running the Kalman Filter on Separate Training and Test Buckets